**Imports**

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

**Devise Congfiguration**

In [2]:
device = 'cpu'

**Hyperparameters**

In [3]:
# we will be treating the image as a sequence
input_size = 28
sequence_len = 28
hidden_size = 200
num_classes = 10
num_epochs = 8
batch_size = 100
learning_rate = 0.001
num_layers = 2

**MNIST Dataset**

In [4]:
train_data = torchvision.datasets.MNIST(root='/content/sample_data', train=True, transform=transforms.ToTensor(), download=True)
test_data = torchvision.datasets.MNIST(root='/content/sample_data', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

Extracting /content/sample_data/MNIST/raw/train-images-idx3-ubyte.gz to /content/sample_data/MNIST/raw


Extracting /content/sample_data/MNIST/raw/train-labels-idx1-ubyte.gz to /content/sample_data/MNIST/raw



Extracting /content/sample_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/sample_data/MNIST/raw


Extracting /content/sample_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/sample_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


**Neural Network**

In [5]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)  # we can also use GRU and LSTM here by using nn.GRU (without any other change) and nn.LSTM (with additionally providing a cell state same as h0)
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

    out, _ = self.rnn(x, h0)
    out = out[:, -1, :]
    out = self.fc(out)
    return out

model = RNN(input_size, hidden_size, num_layers, num_classes)

**Loss and Optimizer**

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

**Training Loops**

In [7]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # resizing the tensors
    images = images.reshape(-1, sequence_len, input_size)

    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%100==0:
      print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

epoch 1 / 8, step 100/600, loss = 0.7516
epoch 1 / 8, step 200/600, loss = 0.4288
epoch 1 / 8, step 300/600, loss = 0.4673
epoch 1 / 8, step 400/600, loss = 0.5796
epoch 1 / 8, step 500/600, loss = 0.2491
epoch 1 / 8, step 600/600, loss = 0.4031
epoch 2 / 8, step 100/600, loss = 0.1854
epoch 2 / 8, step 200/600, loss = 0.2446
epoch 2 / 8, step 300/600, loss = 0.3950
epoch 2 / 8, step 400/600, loss = 0.2766
epoch 2 / 8, step 500/600, loss = 0.1196
epoch 2 / 8, step 600/600, loss = 0.2153
epoch 3 / 8, step 100/600, loss = 0.4631
epoch 3 / 8, step 200/600, loss = 0.2236
epoch 3 / 8, step 300/600, loss = 0.1952
epoch 3 / 8, step 400/600, loss = 0.2638
epoch 3 / 8, step 500/600, loss = 0.1959
epoch 3 / 8, step 600/600, loss = 0.1325
epoch 4 / 8, step 100/600, loss = 0.0829
epoch 4 / 8, step 200/600, loss = 0.1559
epoch 4 / 8, step 300/600, loss = 0.1206
epoch 4 / 8, step 400/600, loss = 0.1402
epoch 4 / 8, step 500/600, loss = 0.1369
epoch 4 / 8, step 600/600, loss = 0.2300
epoch 5 / 8, ste

**Testing and Evaluation**

In [8]:
with torch.no_grad(): # to stop tracking we will use torch.no_grad()
  n_correct = 0
  n_samples = 0
  for images, labels in test_loader:
    images = images.reshape(-1, sequence_len, input_size)
    outputs = model(images)

    _, predictions = torch.max(outputs, 1)
    n_samples += labels.shape[0]
    n_correct += (predictions==labels).sum().item()

acc = 100*(n_correct/n_samples)
print("Accuracy on test data is :", acc)

Accuracy on test data is : 96.77
